In [65]:
import pandas as pd
import pinyin
import re

In [33]:
df_poi = pd.read_pickle("utseus-poi-dianping.pk")
df_poi.head()

,name,branch_name,address,telephone,city,latitude,longitude,avg_rating,product_grade,decoration_grade,service_grade,product_score,decoration_score,service_score,review_count,avg_price,photo_count,category
business_id,,,,,,,,,,,,,,,,,,
500000,阿凡提美食娱乐城(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),,虹口区曲阳路775号天山宾馆B1楼,021-65559604,上海,31.288305,121.490440,30.0,1.0,1.0,1.0,6.8,6.3,6.3,482,81.0,140.0,新疆菜
500001,萨莎(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),,徐汇区东平路11号,021-64746628,上海,31.208550,121.449710,40.0,1.0,1.0,1.0,7.3,8.4,7.4,891,149.0,441.0,酒吧
500001,萨莎(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),,徐汇区东平路11号,021-64746628,上海,31.208550,121.449710,40.0,1.0,1.0,1.0,7.3,8.4,7.4,891,149.0,441.0,牛排
500003,1221餐馆(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),,长宁区延安西路1221号,021-62136585,上海,31.211315,121.427780,35.0,1.0,1.0,1.0,7.1,7.4,7.6,369,144.0,182.0,本帮菜
500008,Food Live(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),,浦东新区世纪大道88号金茂君悦大酒店B1楼,021-50491234-8283,上海,31.235252,121.505745,30.0,1.0,1.0,1.0,6.1,6.3,6.3,38,26.0,12.0,快餐简餐


# Clean data

Only italian restaurants

In [34]:
df_italian = df_poi[df_poi.category == '意大利菜']

Only in Shanghai

In [35]:
df_italian = df_italian[df_italian.city == '上海']

In [36]:
del df_italian['branch_name']

In [37]:
df_italian = df_italian.dropna()

Only with ratings and price info

In [38]:
df_italian = df_italian[(df_italian.avg_price > 0) & (df_italian.avg_rating > 0)]

Clean name

In [39]:
df_italian["name"]= df_italian["name"].str.split("\(这是", n = 1, expand = True) 

Avg_rating /5

In [40]:
df_italian["avg_rating"] = df_italian["avg_rating"]/10

In [41]:
df_italian.head()

,name,address,telephone,city,latitude,longitude,avg_rating,product_grade,decoration_grade,service_grade,product_score,decoration_score,service_score,review_count,avg_price,photo_count,category
business_id,,,,,,,,,,,,,,,,,
500018,华万意意大利餐厅,卢湾区太仓路181弄新天地北里7号楼,021-63112211,上海,31.219019,121.474884,4.0,1.0,1.0,1.0,7.7,8.4,8.0,470,323.0,450.0,意大利菜
500030,艾迪多慕思,静安区延安西路200号文艺宾馆新楼1楼,021-62488499,上海,31.219666,121.441730,4.0,1.0,1.0,1.0,7.7,8.3,7.9,96,354.0,48.0,意大利菜
500123,李奥纳多达芬奇餐厅,静安区华山路250号希尔顿酒店1楼,021-62480000-1850,上海,31.219276,121.445450,4.0,1.0,1.0,1.0,7.7,8.5,8.2,458,296.0,654.0,意大利菜
500137,大马可意大利餐厅,长宁区东诸安浜路103号维景酒店公寓1楼,021-62104495,上海,31.218105,121.434270,4.0,1.0,1.0,1.0,8.3,8.2,7.9,2052,146.0,1996.0,意大利菜
500281,吉范尼斯,长宁区遵义南路5号虹桥喜来登上海太平洋大饭店27楼,021-62758888,上海,31.203594,121.405530,4.0,1.0,1.0,1.0,7.9,8.6,8.8,210,244.0,559.0,意大利菜


# Geometry

In [42]:
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry
import geopandas as gpd

In [43]:
geometry = [Point(xy) for xy in zip(df_italian['longitude'], df_italian['latitude'])]

In [44]:
df_italian['geometry'] = geometry

In [45]:
df_italian.head()

,name,address,telephone,city,latitude,longitude,avg_rating,product_grade,decoration_grade,service_grade,product_score,decoration_score,service_score,review_count,avg_price,photo_count,category,geometry
business_id,,,,,,,,,,,,,,,,,,
500018,华万意意大利餐厅,卢湾区太仓路181弄新天地北里7号楼,021-63112211,上海,31.219019,121.474884,4.0,1.0,1.0,1.0,7.7,8.4,8.0,470,323.0,450.0,意大利菜,POINT (121.474884 31.219019)
500030,艾迪多慕思,静安区延安西路200号文艺宾馆新楼1楼,021-62488499,上海,31.219666,121.441730,4.0,1.0,1.0,1.0,7.7,8.3,7.9,96,354.0,48.0,意大利菜,POINT (121.44173 31.219666)
500123,李奥纳多达芬奇餐厅,静安区华山路250号希尔顿酒店1楼,021-62480000-1850,上海,31.219276,121.445450,4.0,1.0,1.0,1.0,7.7,8.5,8.2,458,296.0,654.0,意大利菜,POINT (121.44545 31.219276)
500137,大马可意大利餐厅,长宁区东诸安浜路103号维景酒店公寓1楼,021-62104495,上海,31.218105,121.434270,4.0,1.0,1.0,1.0,8.3,8.2,7.9,2052,146.0,1996.0,意大利菜,POINT (121.43427 31.218105)
500281,吉范尼斯,长宁区遵义南路5号虹桥喜来登上海太平洋大饭店27楼,021-62758888,上海,31.203594,121.405530,4.0,1.0,1.0,1.0,7.9,8.6,8.8,210,244.0,559.0,意大利菜,POINT (121.40553 31.203594)


In [46]:
df_italian.to_csv('map_italian.csv')

# Add Subway 

In [47]:
df_subway = df_poi[df_poi.category == '地铁/轻轨']

In [50]:
df_subway["name"] = df_subway["name"].str.split("\(这是", n = 1, expand = True) 

C:\Users\Aurelie\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
df_subway.head()

,name,branch_name,address,telephone,city,latitude,longitude,avg_rating,product_grade,decoration_grade,service_grade,product_score,decoration_score,service_score,review_count,avg_price,photo_count,category
business_id,,,,,,,,,,,,,,,,,,
2057173,轨道交通4号线,,徐汇区宜山路 宜山路 环线,021-64370000,上海,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,1375,4.0,313.0,地铁/轻轨
2057596,轨道交通3号线,,徐汇区上海南站－江杨北路,021-64370000,上海,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,1117,4.0,289.0,地铁/轻轨
2058700,轨道交通2号线,,长宁区徐泾东－广兰路－浦东国际机场,021-64370000,上海,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,1456,4.0,461.0,地铁/轻轨
2061139,轨道交通1号线,,闵行区莘庄－富锦路,021-64370000,上海,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,1575,4.0,454.0,地铁/轻轨
2062099,轨道交通5号线,,闵行区莘庄－闵行开发区,021-64370000,上海,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,326,4.0,74.0,地铁/轻轨


Clean

In [52]:
df_subway = df_subway[['name', 'longitude', 'latitude']]

In [58]:
df_subway = df_subway[(df_subway.longitude > 0) & (df_subway.latitude > 0)]

In [102]:
df_subway['name_pinyin'] = df_subway['name'].apply(lambda x: re.sub('((dìtiě)|(zhàn)|(hàoxiàn)|[（）()/\d])*','', pinyin.get(x)).strip())

C:\Users\Aurelie\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
df_subway = df_subway.drop(2069821)

In [127]:
geometry = [Point(xy) for xy in zip(df_subway['longitude'], df_subway['latitude'])]

In [128]:
df_subway['geometry'] = geometry

In [129]:
df_subway

,name,longitude,latitude,name_pinyin,geometry
business_id,,,,,
2071237,地铁通道广告,121.481300,31.243450,tōngdàoguǎnggào,POINT (121.4813 31.24345)
2075992,上海磁浮列车,121.801040,31.148770,shànghǎicífúlièchē,POINT (121.80104 31.14877)
2199364,陆家嘴地铁站,121.501980,31.238090,lùjiāzǔi,POINT (121.50198 31.23809)
2445064,京沪快线,121.349150,31.192700,jīnghùkuàixiàn,POINT (121.34915 31.1927)
4710716,虹口足球场地铁站,121.479530,31.270570,hóngkǒuzúqíucháng,POINT (121.47953 31.27057)
4740839,八号线老西门站,121.485310,31.219645,bālǎoxīmén,POINT (121.48531 31.219645)
4740932,二号线中山公园站,121.408500,31.215150,èrzhōngshāngōngyuán,POINT (121.4085 31.21515)
5140311,地铁一号线共康路站,121.446990,31.319110,yīgòngkānglù,POINT (121.44699 31.31911)
5176033,地铁站黄兴路站,121.528030,31.278660,huángxīnglù,POINT (121.52803 31.27866)
